실행 위치: 로컬

In [ ]:
# PYTHONPATH 설정
import sys, os 

BASE_DIR = os.getcwd()
for _ in range(1):
    BASE_DIR = os.path.dirname(BASE_DIR)

sys.path.insert(0, BASE_DIR)
print(sys.path)


# 1. Tool 함수 정의

In [ ]:
from tools.realtime_weather_api_tool.weather_api_tool import WeatherApiTool

weather_api_tool = WeatherApiTool()

In [ ]:
tool_functions = {"get_weather_api": weather_api_tool.get_weather_api}
tool_description = [weather_api_tool.tools_description]

# 2. 추론 수행 (툴 제공)

In [ ]:
messages = [
    {
        "role": "user",
        "content": "지금 제주시 날씨는 어떄? 적합한 야외 활동 추천해줘"
    }
]


In [ ]:
from fastapi_app.app.core.config import settings

vllm_ip_port = settings.VLLM_IP_PORT.rstrip("/")
vllm_api_key = settings.VLLM_API_KEY
vllm_chat_completions_url = f"{vllm_ip_port}/v1/chat/completions"

payload = {
    "model": "openai/gpt-oss-20b",
    "messages": messages,
    "tools": tool_description,
    "tool_choice": "auto",  # 모델이 스스로 판단하여 툴 사용 여부 결정
    "temperature": 0
}

headers={"Content-Type": "application/json", "Authorization": f"Bearer {vllm_api_key}"}

In [ ]:
import httpx 

async with httpx.AsyncClient(timeout=60.0) as client:
    response = await client.post(url=vllm_chat_completions_url, json=payload, headers=headers)
        

# 3. 응답 확인

In [ ]:
import json

if response.status_code == 200:
    model_response = response.json()
    print(json.dumps(model_response, indent=4, ensure_ascii=False))
    arguments = model_response["choices"][0]["message"]["tool_calls"][0]["function"]["arguments"]
    print('-------------------')
    print(json.loads(arguments))
    
else:
    print(f"에러 발생: {response.status_code}")
    print(response.text)

In [ ]:
# 모델이 응답한 내용을 바탕으로 함수명과 변수를 추출

functions = model_response["choices"][0]["message"]["tool_calls"]

tool_call_id = function["id"]
function_name = function["function"]["name"]
func_args = json.loads(function["function"]["arguments"])

print(f"툴 id: {tool_call_id}")
print(f"함수명: {function_name}")
print(f"변수: {func_args}")

# 4. 툴 실행(모델이 전달한 tool 내용을 바탕으로 실행)

In [ ]:
# 함수 실행
tool_exec_rslt = await tool_functions[function_name](**args)
print(tool_exec_rslt)

In [ ]:
# 모델의 응답과 툴 실행 결과를 messages에 담아 요청

model_message = model_response["choices"][0]["message"]
tool_message = {"role": "tool", "content": tool_exec_rslt, "tool_call_id": tool_call_id}

messages.append(model_message)
messages.append(tool_message)

In [ ]:
messages

In [ ]:
payload["messages"] = messages
print(payload)

# 5. 툴 실행 결과를 모델에게 전달

In [ ]:
import httpx 

async with httpx.AsyncClient(timeout=60.0) as client:
    response = await client.post(url=vllm_chat_completions_url, json=payload, headers=headers)

# 6. 최종 응답 확인

In [ ]:

import json

if response.status_code == 200:
    model_response = response.json()
    print(json.dumps(model_response, indent=4, ensure_ascii=False))
    
else:
    print(f"에러 발생: {response.status_code}")
    print(response.text)